In [0]:
storage_account_name = "storage60301575"
storage_account_key = "2hVwV/06JBSdKiOTedzP6WHZpiJSrfZVdwK8psbRCVT8yQKzRponj/dQecMc2DFc/UUG3u/Wocqq+AStijSp5w=="
spark.conf.set(
 f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
 storage_account_key
)
reviews_path = "abfss://processed@storage60301575.dfs.core.windows.net/reviews/"
reviews_df = spark.read.parquet(reviews_path)


In [0]:
reviews_df.printSchema()
display(reviews_df.limit(5))


root
 |-- reviewerID: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- reviewText: string (nullable = true)
 |-- overall: double (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: integer (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- review_year: integer (nullable = true)



reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,review_year
A3DXRN8UPJ6W2N,B008K4PGX4,"Jason Greenwald ""BA in Television""","List(0, 0)","I love this backpack, the laptop compartment fits my 15"" macbook pro great and with the two padding inserts they send with it my laptop doesn't move. The compartments are great and the front compartment even has an impact shield to prevent your Ipad from getting damaged. I would buy it again.",5.0,LOVE IT,1366848000,"04 25, 2013",2013
ASW7XQVURBMH5,B008K4PGX4,"Joerg Wekenborg ""joerg130376""","List(0, 0)","As a professional traveller having good luggage is a must.If you need a bomb- and bulletproof backpack that can withstand the baghandlers on a bad day, here it is.It's pricey yes, but it'll keep your laptop safe and also your tablet.",5.0,Best backpack EVER!,1384819200,"11 19, 2013",2013
A2U1SCO747SUFJ,B008K4PGX4,John Alvarez,"List(7, 7)","The Pelican U100 Elite Storage Backpack is an extreme computer backpack for extreme conditions. This beast weighs 8.4 pounds empty, so it is obviously made to withstand almost anything. The Pelican laptop compartment is waterproof, crush proof, and has room for a decent size laptop (15.50″ long x 10.50″ width x 1.50″ deep). The cloth material is 1000 denier nylon, which is much thicker than most non-military grade backpacks. It’s the best of both worlds, a Pelican case and a very durable backpack.Who would need such a backpack? Anyone who puts their computer at risk. I take my laptop to and from work in a car with a lot of other equipment. My laptop is subject to being knocked around, short falls, or having other equipment tossed on top of it. This backpack would also be great for soldiers, frequent travelers, motorcycle riders, or anyone who takes their computer to inhospitable environments. This has got to be one of the most protective computer backpacks on earth.I stumbled upon this backpack after writing on my website about a laptop sleeve I purchased to protect my MacBook Pro. This laptop sleeve would be sufficient for most people, but I wanted something offering extreme protection. I considered a standard Pelican case (briefcase style), but that would not have room for the other equipment I need to carry with me on a regular basis. Everything pointed to the Pelican U100 backpack.The Pelican U100 backpack is a bit expensive at a retail price of $269.86. I checked Amazon and it was much cheaper around $185. This was more than I am used to paying for a computer backpack, but it would be cheaper than replacing a high-end laptop. I also feel this laptop is a good deal at the price on Amazon.The plastic laptop compartment is padded, but the padding is thin and not the normal Pelican case padding. It is obvious this was needed to keep the dimensions of the compartment reasonably sized. Newer laptops, MacBooks, and “ultrabooks” are thin so Pelican included padded spacers. I have a MacBook Pro 13 and the two spacers for this model laptop may be sufficient, but I still may look at adding padding that is more substantial. I may take padding from another Pelican case and line the compartment with it.In addition to the plastic laptop storage case, the backpack itself has three main storage compartments:1. The front compartment is thin and has room for an iPad just behind the plastic protective face. There is also room for pens, notepads, or other smaller items. There is also a zippered pouch in this area.2. The next compartment is slightly above the compartment above and takes up about 1/3 of the pack height. This compartment can expand to a decent depth, but it will take room from the third and largest compartment.3. The last and largest compartment runs the length of the pack and can expand to hold a decent amount of gear (one or two days clothes).*There is a somewhat hidden compartment on the back of the backpack that is accessed by moving the shoulder straps out of the way. This compartment is not meant to hold mu

In [0]:
from pyspark.sql.functions import col, trim, length
clean_reviews_df = reviews_df
# Drop rows with missing critical fields
clean_reviews_df = clean_reviews_df.filter(
 col("asin").isNotNull() &
 col("reviewerID").isNotNull() &
 col("overall").isNotNull()
)
# Enforce valid rating values
clean_reviews_df = clean_reviews_df.filter(
 (col("overall") >= 1) & (col("overall") <= 5)
)
# Clean review text
clean_reviews_df = clean_reviews_df.withColumn(
 "reviewText",
 trim(col("reviewText"))
).filter(
 length(col("reviewText")) >= 10
)


In [0]:
print("Rows before cleaning:", reviews_df.count())
print("Rows after cleaning:", clean_reviews_df.count())
display(clean_reviews_df.limit(5))

Rows before cleaning: 11824316
Rows after cleaning: 11811989


reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,review_year
A3DXRN8UPJ6W2N,B008K4PGX4,"Jason Greenwald ""BA in Television""","List(0, 0)","I love this backpack, the laptop compartment fits my 15"" macbook pro great and with the two padding inserts they send with it my laptop doesn't move. The compartments are great and the front compartment even has an impact shield to prevent your Ipad from getting damaged. I would buy it again.",5.0,LOVE IT,1366848000,"04 25, 2013",2013
ASW7XQVURBMH5,B008K4PGX4,"Joerg Wekenborg ""joerg130376""","List(0, 0)","As a professional traveller having good luggage is a must.If you need a bomb- and bulletproof backpack that can withstand the baghandlers on a bad day, here it is.It's pricey yes, but it'll keep your laptop safe and also your tablet.",5.0,Best backpack EVER!,1384819200,"11 19, 2013",2013
A2U1SCO747SUFJ,B008K4PGX4,John Alvarez,"List(7, 7)","The Pelican U100 Elite Storage Backpack is an extreme computer backpack for extreme conditions. This beast weighs 8.4 pounds empty, so it is obviously made to withstand almost anything. The Pelican laptop compartment is waterproof, crush proof, and has room for a decent size laptop (15.50″ long x 10.50″ width x 1.50″ deep). The cloth material is 1000 denier nylon, which is much thicker than most non-military grade backpacks. It’s the best of both worlds, a Pelican case and a very durable backpack.Who would need such a backpack? Anyone who puts their computer at risk. I take my laptop to and from work in a car with a lot of other equipment. My laptop is subject to being knocked around, short falls, or having other equipment tossed on top of it. This backpack would also be great for soldiers, frequent travelers, motorcycle riders, or anyone who takes their computer to inhospitable environments. This has got to be one of the most protective computer backpacks on earth.I stumbled upon this backpack after writing on my website about a laptop sleeve I purchased to protect my MacBook Pro. This laptop sleeve would be sufficient for most people, but I wanted something offering extreme protection. I considered a standard Pelican case (briefcase style), but that would not have room for the other equipment I need to carry with me on a regular basis. Everything pointed to the Pelican U100 backpack.The Pelican U100 backpack is a bit expensive at a retail price of $269.86. I checked Amazon and it was much cheaper around $185. This was more than I am used to paying for a computer backpack, but it would be cheaper than replacing a high-end laptop. I also feel this laptop is a good deal at the price on Amazon.The plastic laptop compartment is padded, but the padding is thin and not the normal Pelican case padding. It is obvious this was needed to keep the dimensions of the compartment reasonably sized. Newer laptops, MacBooks, and “ultrabooks” are thin so Pelican included padded spacers. I have a MacBook Pro 13 and the two spacers for this model laptop may be sufficient, but I still may look at adding padding that is more substantial. I may take padding from another Pelican case and line the compartment with it.In addition to the plastic laptop storage case, the backpack itself has three main storage compartments:1. The front compartment is thin and has room for an iPad just behind the plastic protective face. There is also room for pens, notepads, or other smaller items. There is also a zippered pouch in this area.2. The next compartment is slightly above the compartment above and takes up about 1/3 of the pack height. This compartment can expand to a decent depth, but it will take room from the third and largest compartment.3. The last and largest compartment runs the length of the pack and can expand to hold a decent amount of gear (one or two days clothes).*There is a somewhat hidden compartment on the back of the backpack that is accessed by moving the shoulder straps out of the way. This compartment is not meant to hold mu

In [0]:
clean_reviews_path = "abfss://processed@storage60301575.dfs.core.windows.net/clean_reviews/"
clean_reviews_df.write.mode("overwrite").parquet(clean_reviews_path)